# Day 3 - Text-to-SQL with LangGraph
# 자연어를 SQL로 변환하는 시스템

이 실습에서는 LangGraph와 Day 1의 파인튜닝 모델을 사용하여
자연어 질문을 SQL 쿼리로 변환하고 실행하는 시스템을 구현합니다.

## 핵심 개념
- 스키마 이해 및 분석
- 자연어 → SQL 변환
- SQL 검증 및 수정
- 결과 해석 및 자연어 요약

In [ ]:
!pip install langchain langgraph sqlite3 pandas sqlalchemy transformers torch

In [ ]:
import os
import sqlite3
import pandas as pd
import json
from typing import List, Dict, Any, Optional, TypedDict
import re

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

from langchain.schema import Document
from langgraph.graph import StateGraph, END
from sqlalchemy import create_engine, text

print("라이브러리 import 완료")

## 1. Day 1 파인튜닝 모델 로드

In [ ]:
class Day1FinetunedLLM:
    def __init__(self, model_name="ryanu/my-exaone-raft-model"):
        self.model_name = model_name
        self.base_model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"모델 로딩 중: {model_name}")
        print(f"사용 디바이스: {self.device}")
        
        # 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.base_model,
            trust_remote_code=True
        )
        
        # 베이스 모델 로드
        self.model = AutoModelForCausalLM.from_pretrained(
            self.base_model,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )
        
        # 파인튜닝된 어댑터 로드
        try:
            self.model = PeftModel.from_pretrained(self.model, model_name)
            print("✅ 파인튜닝 모델 로드 완료")
        except Exception as e:
            print(f"⚠️ 파인튜닝 모델 로드 실패, 베이스 모델 사용: {e}")
    
    def generate(self, prompt: str, max_length: int = 512, temperature: float = 0.1) -> str:
        """텍스트 생성 (SQL 생성에는 낮은 temperature 사용)"""
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=2048
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=inputs['input_ids'].shape[1] + max_length,
                temperature=temperature,
                do_sample=True if temperature > 0 else False,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        generated_text = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return generated_text.strip()

# 모델 초기화
llm = Day1FinetunedLLM()

## 2. 샘플 데이터베이스 생성

In [ ]:
def create_sample_database():
    """샘플 전자상거래 데이터베이스 생성"""
    # SQLite 연결
    conn = sqlite3.connect('ecommerce.db')
    cursor = conn.cursor()
    
    # 기존 테이블 삭제
    cursor.execute('DROP TABLE IF EXISTS order_items')
    cursor.execute('DROP TABLE IF EXISTS orders')
    cursor.execute('DROP TABLE IF EXISTS products')
    cursor.execute('DROP TABLE IF EXISTS customers')
    cursor.execute('DROP TABLE IF EXISTS categories')
    
    # 카테고리 테이블
    cursor.execute('''
        CREATE TABLE categories (
            category_id INTEGER PRIMARY KEY,
            category_name TEXT NOT NULL,
            description TEXT
        )
    ''')
    
    # 상품 테이블
    cursor.execute('''
        CREATE TABLE products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            category_id INTEGER,
            price DECIMAL(10, 2),
            stock_quantity INTEGER,
            created_date DATE,
            FOREIGN KEY (category_id) REFERENCES categories (category_id)
        )
    ''')
    
    # 고객 테이블
    cursor.execute('''
        CREATE TABLE customers (
            customer_id INTEGER PRIMARY KEY,
            customer_name TEXT NOT NULL,
            email TEXT,
            city TEXT,
            join_date DATE
        )
    ''')
    
    # 주문 테이블
    cursor.execute('''
        CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            order_date DATE,
            total_amount DECIMAL(10, 2),
            status TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
        )
    ''')
    
    # 주문 상품 테이블
    cursor.execute('''
        CREATE TABLE order_items (
            order_item_id INTEGER PRIMARY KEY,
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            unit_price DECIMAL(10, 2),
            FOREIGN KEY (order_id) REFERENCES orders (order_id),
            FOREIGN KEY (product_id) REFERENCES products (product_id)
        )
    ''')
    
    # 샘플 데이터 삽입
    # 카테고리 데이터
    categories_data = [
        (1, '전자제품', '컴퓨터, 스마트폰 등'),
        (2, '의류', '남성복, 여성복, 아동복'),
        (3, '도서', '소설, 전문서적, 만화'),
        (4, '가전제품', '냉장고, 세탁기, TV')
    ]
    cursor.executemany('INSERT INTO categories VALUES (?, ?, ?)', categories_data)
    
    # 상품 데이터
    products_data = [
        (1, '아이폰 15', 1, 1200000, 50, '2024-01-15'),
        (2, '갤럭시 S24', 1, 1100000, 30, '2024-01-20'),
        (3, '맥북 프로', 1, 2500000, 15, '2024-02-01'),
        (4, '나이키 운동화', 2, 150000, 100, '2024-01-10'),
        (5, '데이터 사이언스 교과서', 3, 45000, 200, '2024-01-05'),
        (6, 'LG 냉장고', 4, 800000, 20, '2024-02-10'),
        (7, '아디다스 티셔츠', 2, 35000, 150, '2024-01-25'),
        (8, '파이썬 프로그래밍', 3, 38000, 80, '2024-01-30')
    ]
    cursor.executemany('INSERT INTO products VALUES (?, ?, ?, ?, ?, ?)', products_data)
    
    # 고객 데이터
    customers_data = [
        (1, '김철수', 'kim@email.com', '서울', '2023-01-15'),
        (2, '이영희', 'lee@email.com', '부산', '2023-03-20'),
        (3, '박민수', 'park@email.com', '대구', '2023-05-10'),
        (4, '최지은', 'choi@email.com', '서울', '2023-07-25'),
        (5, '정다영', 'jung@email.com', '인천', '2023-09-15')
    ]
    cursor.executemany('INSERT INTO customers VALUES (?, ?, ?, ?, ?)', customers_data)
    
    # 주문 데이터
    orders_data = [
        (1, 1, '2024-01-20', 1200000, '배송완료'),
        (2, 2, '2024-01-25', 185000, '배송중'),
        (3, 1, '2024-02-01', 83000, '배송완료'),
        (4, 3, '2024-02-05', 2500000, '배송완료'),
        (5, 4, '2024-02-10', 1150000, '배송준비중'),
        (6, 2, '2024-02-15', 800000, '주문접수'),
        (7, 5, '2024-02-20', 70000, '배송완료')
    ]
    cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?, ?)', orders_data)
    
    # 주문 상품 데이터
    order_items_data = [
        (1, 1, 1, 1, 1200000),  # 김철수 - 아이폰
        (2, 2, 4, 1, 150000),   # 이영희 - 나이키 운동화
        (3, 2, 7, 1, 35000),    # 이영희 - 아디다스 티셔츠
        (4, 3, 5, 1, 45000),    # 김철수 - 데이터 사이언스 교과서
        (5, 3, 8, 1, 38000),    # 김철수 - 파이썬 프로그래밍
        (6, 4, 3, 1, 2500000),  # 박민수 - 맥북 프로
        (7, 5, 2, 1, 1100000),  # 최지은 - 갤럭시
        (8, 5, 4, 1, 150000),   # 최지은 - 나이키 운동화
        (9, 6, 6, 1, 800000),   # 이영희 - LG 냉장고
        (10, 7, 7, 2, 35000)    # 정다영 - 아디다스 티셔츠 2개
    ]
    cursor.executemany('INSERT INTO order_items VALUES (?, ?, ?, ?, ?)', order_items_data)
    
    conn.commit()
    conn.close()
    
    print("✅ 샘플 전자상거래 데이터베이스 생성 완료")
    return 'ecommerce.db'

# 데이터베이스 생성
db_path = create_sample_database()

## 3. 데이터베이스 스키마 분석기

In [ ]:
class DatabaseAnalyzer:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.engine = create_engine(f'sqlite:///{db_path}')
        self.schema_info = self._analyze_schema()
    
    def _analyze_schema(self) -> Dict[str, Any]:
        """데이터베이스 스키마 분석"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 테이블 목록 가져오기
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        
        schema_info = {
            'tables': {},
            'relationships': [],
            'sample_data': {}
        }
        
        for table in tables:
            # 컬럼 정보 가져오기
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            
            schema_info['tables'][table] = {
                'columns': [
                    {
                        'name': col[1],
                        'type': col[2],
                        'nullable': not col[3],
                        'primary_key': bool(col[5])
                    }
                    for col in columns
                ]
            }
            
            # 외래 키 정보
            cursor.execute(f"PRAGMA foreign_key_list({table})")
            foreign_keys = cursor.fetchall()
            
            for fk in foreign_keys:
                schema_info['relationships'].append({
                    'from_table': table,
                    'from_column': fk[3],
                    'to_table': fk[2],
                    'to_column': fk[4]
                })
            
            # 샘플 데이터
            cursor.execute(f"SELECT * FROM {table} LIMIT 3")
            sample_rows = cursor.fetchall()
            schema_info['sample_data'][table] = sample_rows
        
        conn.close()
        return schema_info
    
    def get_schema_description(self) -> str:
        """스키마 설명 텍스트 생성"""
        description = "데이터베이스 스키마 정보:\n\n"
        
        for table_name, table_info in self.schema_info['tables'].items():
            description += f"테이블: {table_name}\n"
            description += "컬럼:\n"
            
            for col in table_info['columns']:
                pk_marker = " (PK)" if col['primary_key'] else ""
                nullable_marker = " (NULL 허용)" if col['nullable'] else ""
                description += f"  - {col['name']}: {col['type']}{pk_marker}{nullable_marker}\n"
            
            description += "\n"
        
        if self.schema_info['relationships']:
            description += "관계:\n"
            for rel in self.schema_info['relationships']:
                description += f"  - {rel['from_table']}.{rel['from_column']} → {rel['to_table']}.{rel['to_column']}\n"
        
        return description
    
    def execute_query(self, query: str) -> pd.DataFrame:
        """SQL 쿼리 실행"""
        try:
            return pd.read_sql_query(query, self.engine)
        except Exception as e:
            raise Exception(f"SQL 실행 오류: {str(e)}")

# 데이터베이스 분석기 초기화
db_analyzer = DatabaseAnalyzer(db_path)
print("✅ 데이터베이스 분석기 초기화 완료")
print("\n스키마 정보:")
print(db_analyzer.get_schema_description())

## 4. Text2SQL 상태 정의

In [ ]:
class Text2SQLState(TypedDict):
    """Text2SQL 워크플로 상태"""
    question: str              # 자연어 질문
    schema_context: str        # 스키마 정보
    generated_sql: str         # 생성된 SQL
    sql_valid: bool           # SQL 유효성
    sql_error: str            # SQL 오류 메시지
    query_result: pd.DataFrame # 쿼리 결과
    natural_answer: str        # 자연어 답변
    confidence: float          # 신뢰도
    need_revision: bool        # 수정 필요 여부
    revision_count: int        # 수정 횟수

print("✅ Text2SQL 상태 클래스 정의 완료")

## 5. Text2SQL 워크플로 노드들

In [ ]:
def analyze_question(state: Text2SQLState) -> Text2SQLState:
    """질문 분석 및 스키마 컨텍스트 준비"""
    question = state["question"]
    
    print(f"📋 질문 분석: {question}")
    
    # 스키마 정보를 컨텍스트로 준비
    schema_context = db_analyzer.get_schema_description()
    
    return {
        **state,
        "schema_context": schema_context,
        "revision_count": state.get("revision_count", 0)
    }

def generate_sql(state: Text2SQLState) -> Text2SQLState:
    """자연어 질문을 SQL로 변환"""
    question = state["question"]
    schema_context = state["schema_context"]
    
    print("🔄 SQL 생성 중...")
    
    # SQL 생성 프롬프트
    prompt = f"""당신은 전문적인 SQL 개발자입니다. 주어진 데이터베이스 스키마를 바탕으로 자연어 질문을 정확한 SQL 쿼리로 변환해주세요.

데이터베이스 스키마:
{schema_context}

자연어 질문: {question}

요구사항:
1. SQLite 문법을 사용하세요
2. 정확한 테이블명과 컬럼명을 사용하세요
3. 적절한 JOIN을 사용하세요
4. 가독성 있는 SQL을 작성하세요
5. SQL 쿼리만 반환하고 설명은 포함하지 마세요

SQL 쿼리:"""
    
    generated_sql = llm.generate(prompt, max_length=300, temperature=0.1)
    
    # SQL 쿼리 정제 (주석이나 설명 제거)
    sql_lines = []
    for line in generated_sql.split('\n'):
        line = line.strip()
        if line and not line.startswith('--') and not line.startswith('#'):
            sql_lines.append(line)
    
    clean_sql = ' '.join(sql_lines).strip()
    
    print(f"🔍 생성된 SQL: {clean_sql}")
    
    return {
        **state,
        "generated_sql": clean_sql
    }

def validate_sql(state: Text2SQLState) -> Text2SQLState:
    """생성된 SQL 유효성 검증"""
    sql = state["generated_sql"]
    
    print("✅ SQL 유효성 검증 중...")
    
    try:
        # SQL 실행 시도
        result = db_analyzer.execute_query(sql)
        
        # 결과가 비어있지 않은지 확인
        is_valid = True
        error_msg = ""
        
        if result.empty:
            print("⚠️ 쿼리 결과가 비어있습니다")
        
        print(f"✅ SQL 검증 성공, {len(result)}개 행 반환")
        
    except Exception as e:
        is_valid = False
        error_msg = str(e)
        result = pd.DataFrame()
        print(f"❌ SQL 검증 실패: {error_msg}")
    
    return {
        **state,
        "sql_valid": is_valid,
        "sql_error": error_msg,
        "query_result": result
    }

def fix_sql(state: Text2SQLState) -> Text2SQLState:
    """SQL 오류 수정"""
    question = state["question"]
    schema_context = state["schema_context"]
    broken_sql = state["generated_sql"]
    error_msg = state["sql_error"]
    
    print("🔧 SQL 오류 수정 중...")
    
    # 오류 수정 프롬프트
    prompt = f"""다음 SQL 쿼리에 오류가 있습니다. 오류를 분석하고 올바른 SQL로 수정해주세요.

데이터베이스 스키마:
{schema_context}

원래 질문: {question}
오류가 있는 SQL: {broken_sql}
오류 메시지: {error_msg}

요구사항:
1. 오류를 정확히 분석하세요
2. 스키마에 맞는 올바른 테이블명과 컬럼명을 사용하세요
3. SQLite 문법을 준수하세요
4. 수정된 SQL 쿼리만 반환하세요

수정된 SQL:"""
    
    fixed_sql = llm.generate(prompt, max_length=300, temperature=0.1)
    
    # SQL 정제
    sql_lines = []
    for line in fixed_sql.split('\n'):
        line = line.strip()
        if line and not line.startswith('--') and not line.startswith('#'):
            sql_lines.append(line)
    
    clean_sql = ' '.join(sql_lines).strip()
    
    print(f"🔧 수정된 SQL: {clean_sql}")
    
    return {
        **state,
        "generated_sql": clean_sql,
        "revision_count": state["revision_count"] + 1
    }

def interpret_results(state: Text2SQLState) -> Text2SQLState:
    """쿼리 결과를 자연어로 해석"""
    question = state["question"]
    sql = state["generated_sql"]
    result = state["query_result"]
    
    print("📊 결과 해석 중...")
    
    # 결과 요약 생성
    if result.empty:
        natural_answer = "검색 조건에 맞는 데이터가 없습니다."
        confidence = 0.7
    else:
        # 결과 데이터를 텍스트로 변환
        result_text = result.to_string(index=False, max_rows=10)
        
        # 자연어 답변 생성 프롬프트
        prompt = f"""다음 SQL 쿼리 결과를 바탕으로 원래 질문에 대한 자연어 답변을 작성해주세요.

원래 질문: {question}
실행된 SQL: {sql}
쿼리 결과:
{result_text}

요구사항:
1. 결과를 이해하기 쉽게 설명하세요
2. 구체적인 숫자나 값을 포함하세요
3. 질문에 직접적으로 답변하세요
4. 간결하고 명확하게 작성하세요

자연어 답변:"""
        
        natural_answer = llm.generate(prompt, max_length=200, temperature=0.3)
        
        # 신뢰도 계산 (간단한 휴리스틱)
        if len(result) > 0:
            confidence = min(0.9, 0.6 + len(result.columns) * 0.1)
        else:
            confidence = 0.5
    
    print(f"💬 자연어 답변: {natural_answer}")
    print(f"📊 신뢰도: {confidence:.3f}")
    
    return {
        **state,
        "natural_answer": natural_answer,
        "confidence": confidence
    }

def evaluate_response(state: Text2SQLState) -> Text2SQLState:
    """응답 품질 평가"""
    confidence = state["confidence"]
    revision_count = state["revision_count"]
    sql_valid = state["sql_valid"]
    
    # 수정이 필요한 조건
    need_revision = (
        not sql_valid and           # SQL이 유효하지 않고
        revision_count < 2 and      # 수정 횟수가 2회 미만이고
        confidence < 0.6            # 신뢰도가 낮음
    )
    
    print(f"📊 응답 평가: 유효={sql_valid}, 신뢰도={confidence:.3f}, 수정={revision_count}, 재수정필요={need_revision}")
    
    return {
        **state,
        "need_revision": need_revision
    }

def should_revise(state: Text2SQLState) -> str:
    """수정 여부 결정"""
    if state["need_revision"]:
        return "fix"
    else:
        return "end"

print("✅ Text2SQL 워크플로 노드 함수 정의 완료")

## 6. LangGraph Text2SQL 워크플로 구성

In [ ]:
# Text2SQL 워크플로 생성
workflow = StateGraph(Text2SQLState)

# 노드 추가
workflow.add_node("analyze", analyze_question)
workflow.add_node("generate", generate_sql)
workflow.add_node("validate", validate_sql)
workflow.add_node("fix", fix_sql)
workflow.add_node("interpret", interpret_results)
workflow.add_node("evaluate", evaluate_response)

# 엣지 연결
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "generate")
workflow.add_edge("generate", "validate")
workflow.add_edge("validate", "interpret")
workflow.add_edge("interpret", "evaluate")

# 조건부 엣지
workflow.add_conditional_edges(
    "evaluate",
    should_revise,
    {
        "fix": "fix",
        "end": END
    }
)

# 수정 후 다시 검증으로
workflow.add_edge("fix", "validate")

# 앱 컴파일
text2sql_app = workflow.compile()

print("✅ Text2SQL LangGraph 워크플로 구성 완료")

## 7. Text2SQL 시스템 테스트

In [ ]:
def run_text2sql(question: str):
    """Text2SQL 시스템 실행"""
    print(f"\n🚀 Text2SQL 시스템 시작")
    print(f"❓ 질문: {question}")
    print("=" * 80)
    
    # 초기 상태
    initial_state = {
        "question": question,
        "revision_count": 0
    }
    
    # 워크플로 실행
    final_state = text2sql_app.invoke(initial_state)
    
    print("\n📊 최종 결과")
    print("=" * 80)
    print(f"🔍 생성된 SQL: {final_state['generated_sql']}")
    print(f"✅ SQL 유효성: {final_state['sql_valid']}")
    print(f"💬 자연어 답변: {final_state['natural_answer']}")
    print(f"📊 신뢰도: {final_state['confidence']:.3f}")
    print(f"🔄 수정 횟수: {final_state['revision_count']}")
    
    # 쿼리 결과 표시
    if not final_state['query_result'].empty:
        print("\n📋 쿼리 결과:")
        print(final_state['query_result'].to_string(index=False))
    
    # 오류 정보 표시
    if final_state.get('sql_error'):
        print(f"\n❌ SQL 오류: {final_state['sql_error']}")
    
    return final_state

# 테스트 질문들
test_questions = [
    "가장 비싼 상품은 무엇인가요?",
    "서울에 사는 고객들의 총 주문 금액은?",
    "전자제품 카테고리의 상품들을 보여주세요",
    "각 고객별 주문 횟수를 알려주세요",
    "배송완료된 주문의 평균 금액은?"
]

# 첫 번째 질문으로 테스트
result = run_text2sql(test_questions[0])

## 8. 다양한 질문으로 성능 테스트

In [ ]:
# 모든 테스트 질문 실행
results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n\n🧪 테스트 {i}/{len(test_questions)}")
    try:
        result = run_text2sql(question)
        results.append({
            "question": question,
            "sql": result["generated_sql"],
            "valid": result["sql_valid"],
            "answer": result["natural_answer"],
            "confidence": result["confidence"],
            "revisions": result["revision_count"]
        })
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        results.append({
            "question": question,
            "sql": "오류 발생",
            "valid": False,
            "answer": f"오류: {str(e)}",
            "confidence": 0.0,
            "revisions": 0
        })

# 결과 요약
print("\n\n📊 전체 테스트 결과 요약")
print("=" * 100)

valid_results = [r for r in results if r["valid"]]
success_rate = len(valid_results) / len(results) * 100
avg_confidence = sum(r["confidence"] for r in valid_results) / max(len(valid_results), 1)
avg_revisions = sum(r["revisions"] for r in results) / len(results)

print(f"성공률: {success_rate:.1f}% ({len(valid_results)}/{len(results)})")
print(f"평균 신뢰도: {avg_confidence:.3f}")
print(f"평균 수정 횟수: {avg_revisions:.1f}")

print("\n📋 상세 결과:")
for i, result in enumerate(results, 1):
    status = "✅" if result["valid"] else "❌"
    print(f"\n[{i}] {status} {result['question']}")
    print(f"    SQL: {result['sql'][:60]}...")
    print(f"    답변: {result['answer'][:80]}...")
    print(f"    신뢰도: {result['confidence']:.3f}, 수정: {result['revisions']}회")

## 9. 고급 기능: 복잡한 질문 처리

In [ ]:
# 더 복잡한 질문들로 테스트
complex_questions = [
    "각 카테고리별 평균 주문 금액과 주문 건수를 보여주세요",
    "2024년 2월에 가장 많이 팔린 상품 top 3는?",
    "고객별 총 구매 금액과 주문 횟수를 내림차순으로 정렬해주세요",
    "배송 상태별 주문 건수와 총 금액을 알려주세요",
    "전자제품을 구매한 고객들의 평균 구매 금액은?"
]

print("🔬 복잡한 질문 테스트")
print("=" * 50)

for i, question in enumerate(complex_questions[:3], 1):  # 처음 3개만 테스트
    print(f"\n🧪 복잡한 질문 {i}")
    try:
        result = run_text2sql(question)
        print(f"✅ 처리 완료: 신뢰도 {result['confidence']:.3f}")
    except Exception as e:
        print(f"❌ 처리 실패: {e}")

## 10. 성능 분석 및 개선 방안

In [ ]:
def analyze_text2sql_performance():
    """Text2SQL 성능 분석"""
    print("🔬 Text2SQL 시스템 성능 분석")
    print("=" * 50)
    
    # 성공/실패 분석
    total_questions = len(results)
    successful = sum(1 for r in results if r["valid"])
    failed = total_questions - successful
    
    print(f"전체 질문: {total_questions}개")
    print(f"성공: {successful}개 ({successful/total_questions*100:.1f}%)")
    print(f"실패: {failed}개 ({failed/total_questions*100:.1f}%)")
    
    # 신뢰도 분포
    if successful > 0:
        confidences = [r["confidence"] for r in results if r["valid"]]
        high_conf = sum(1 for c in confidences if c >= 0.8)
        medium_conf = sum(1 for c in confidences if 0.6 <= c < 0.8)
        low_conf = sum(1 for c in confidences if c < 0.6)
        
        print("\n신뢰도 분포 (성공한 경우):")
        print(f"  높음 (≥0.8): {high_conf}개")
        print(f"  보통 (0.6~0.8): {medium_conf}개")
        print(f"  낮음 (<0.6): {low_conf}개")
    
    # 수정 횟수 분석
    revision_counts = [r["revisions"] for r in results]
    no_revision = sum(1 for r in revision_counts if r == 0)
    one_revision = sum(1 for r in revision_counts if r == 1)
    multi_revision = sum(1 for r in revision_counts if r >= 2)
    
    print("\n수정 횟수 분포:")
    print(f"  수정 없음: {no_revision}개")
    print(f"  1회 수정: {one_revision}개")
    print(f"  2회 이상: {multi_revision}개")
    
    print("\n🎯 시스템 장점:")
    print("  - Day 1 파인튜닝 모델의 SQL 생성 능력 활용")
    print("  - 자동 스키마 분석 및 컨텍스트 제공")
    print("  - SQL 검증 및 자동 수정 기능")
    print("  - 쿼리 결과의 자연어 해석")
    
    print("\n💡 개선 제안:")
    if success_rate < 80:
        print("  - 더 정교한 스키마 설명 및 예시 추가")
        print("  - SQL 생성 프롬프트 개선")
    if avg_revisions > 0.5:
        print("  - 초기 SQL 생성 품질 향상")
        print("  - 더 나은 오류 분석 및 수정 로직")
    if avg_confidence < 0.7:
        print("  - 신뢰도 계산 방식 개선")
        print("  - 결과 검증 로직 강화")

analyze_text2sql_performance()

## 11. 실습 과제

### 과제 1: 스키마 설명 개선
- 각 테이블의 비즈니스 의미와 관계를 더 자세히 설명하는 기능을 추가해보세요
- 샘플 데이터를 포함한 더 풍부한 컨텍스트를 제공해보세요

### 과제 2: 쿼리 최적화
- 생성된 SQL의 성능을 분석하고 최적화하는 노드를 추가해보세요
- 인덱스 활용 여부를 확인하는 기능을 구현해보세요

### 과제 3: 다중 쿼리 처리
- 한 번의 질문에 여러 개의 SQL이 필요한 경우를 처리해보세요
- 복잡한 분석을 위한 단계별 쿼리 실행을 구현해보세요

### 과제 4: 실시간 데이터베이스 연동
- PostgreSQL이나 MySQL 같은 실제 데이터베이스와 연동해보세요
- 더 큰 스키마와 복잡한 관계를 다루는 시스템을 구축해보세요

In [ ]:
# 정리 및 다음 단계
print("🎓 Text2SQL 실습 완료!")
print("\n다음 단계:")
print("1. 05-mcp-integration.ipynb - MCP(Model Context Protocol) 통합")
print("2. 06-gradio-ui.ipynb - Gradio를 통한 UI 래핑")

print("\n💡 핵심 학습 내용:")
print("- 자연어를 SQL로 변환하는 완전한 워크플로")
print("- 데이터베이스 스키마 자동 분석 및 활용")
print("- SQL 검증 및 오류 자동 수정")
print("- 쿼리 결과의 자연어 해석")
print("- Day 1 파인튜닝 모델의 구조화된 출력 생성")

# 리소스 정리
db_analyzer.engine.dispose()
print("\n✅ 리소스 정리 완료")